In [93]:
import bs4
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import re

url = 'https://store.musinsa.com/app/items/lists/001001'
htp = 'https://store.musinsa.com/'
res = requests.get(url)
html = res.text
bs = BeautifulSoup(html, 'html.parser')
tag_lists = bs.select('.li_box .li_inner')

page_lists = []
for tag in tag_lists:
    dict_lists = {}

    # 개별 페이지 링크 파싱
    tmp_lists = tag.select('.list_img a')
    for tmp in tmp_lists:
        link = urljoin(htp, tmp['href'])
        dict_lists['link'] = link
    # 이름 파싱
    article_lists = tag.select('.article_info .list_info a')
    for article in article_lists:
        #print(name.text.strip())
        dict_lists['article'] = article.text.strip()
    page_lists.append(dict_lists)


In [167]:
# 상세 페이지 검색
cloth_lists = []
for page in page_lists[0:5]:
    link  = page['link']
    d_res = requests.get(link)
    d_html = d_res.text
    d_bs = BeautifulSoup(d_html, 'html.parser')
    cloth_dict = {}
    
    # 옷 이름 파싱
    title = d_bs.select('div.right_contents span.product_title > span')[0].text
    cloth_dict['title'] = title
    
    # season, gender 태그 파싱
    detail_lists = d_bs.select('ul.product_article li p.product_article_contents')
    detail_list = []
    for idx, detail in enumerate(detail_lists[1]):
        detail_dict = {}
        if isinstance(detail, bs4.element.Tag):
            content = detail.text
            content = content.replace(' ','')
            content = content.replace('\t', '')
            matched = re.search(r'(\D+)', content)
            val = matched.group(1)
            # print(idx, val.strip())
            if idx == 1:
                cloth_dict['season'] = val.strip()
            elif idx == 5:
                cloth_dict['gender'] = val.strip()
            else:
                continue
 
    # 옷 색깔 파싱
    color = d_bs.select('div.product_info_table > table > tbody > tr > td')[0].text.strip()
    cloth_dict['color'] = color
    
    # 옷 정보 리스트에 추가
    cloth_lists.append(cloth_dict)

cloth_lists

[{'title': '[패키지] NYC LOCATION T-SHIRT (10 COLORS) [GTS721I23]',
  'season': 'S/S',
  'gender': '남',
  'color': '기타'},
 {'title': '릴렉스 핏 크루 넥 반팔 티셔츠 [블랙]',
  'season': 'ALL',
  'gender': '남',
  'color': '검정색'},
 {'title': '릴렉스 핏 크루 넥 반팔 티셔츠 [화이트]',
  'season': 'ALL',
  'gender': '남',
  'color': '흰색'},
 {'title': 'LMC THREE BEARS TEE white',
  'season': 'S/S',
  'gender': '남',
  'color': '흰색'},
 {'title': 'S/S AUTHENTIC LOGO TEE WHITE',
  'season': 'S/S',
  'gender': '남',
  'color': '흰색'}]

In [ ]:
# DataFrame 저장
import 